# ### Anaconda & Python Version:

The version of the notebook server is 5.0.0 and is running on:
Python 3.6.6 |Anaconda custom (64-bit)| (default, Jan 16 2018, 10:22:32) [MSC v.1900 64 bit (AMD64)]

Current Kernel Information:
Python 3.6.4 |Anaconda custom (64-bit)| (default, Jan 16 2018, 10:22:32) [MSC v.1900 64 bit (AMD64)]
Type 'copyright', 'credits' or 'license' for more information
IPython 6.1.0 -- An enhanced Interactive Python. Type '?' for help.

#### Anaconda 5.0.1

In [2]:
# Versions of Libraries: (Code: 'pip list' in anaconda prompt)
# Check version of anaconda - 'conda list anaconda$' or 'conda info' in anaconda command prompt
# Check version of python - 'python' in anaconda command prompt

In [ ]:
### Problem: Recommendation System for like/not-like recommendations
### Problem Type: 
### Steps:
#### 0. 
#### 1. 
#### 2. Note: Should have a validation set also
#### 3. 
#### 4. 
#### 5. 
#### 6. 
#### 7.

In [4]:
## install pytorch:
# pytorch - (wrapper library over torch) is easiest and fastest for Boltzmann Machines, compared to tensorflow, keras
# pytorch - developed by Facebook AI research group, good for NLP, DL on GPUs
# keras - (wrapper library over tensorflow/theano), makes DL more intuitive and easy to use but no enhancement in speed

# installing pytorch:
# go to: https://pytorch.org/; select Stable, Windows, Pip, 3.6, None;
# change 'pip3' to 'pip' in below link:
# run "pip install http://download.pytorch.org/whl/cpu/torch-0.4.1-cp36-cp36m-win_amd64.whl" in anaconda cmd
# succefully installs: torch-0.4.1            # although author installed 'torch 0.1.10.post1'

In [5]:
## standard datasets for movie recommendation system:
# link - https://grouplens.org/datasets/movielens/
# for this problem, download 'ml-100k.zip' and 'ml-1m.zip' (same on grouplens and superdatascience)

In [6]:
## Special Note: Here we are going to work on torch tensors and not tensorflow tensors

In [ ]:
try:
    import torch
except ImportError as e:
    !pip install torch

In [7]:
## import libraries:
import numpy as np
import pandas as pd
# import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [8]:
## import dataset:
movies = pd.read_csv('C:/Users/anurag.trivedi/2. Data/5. Boltzmann Machines/Boltzmann_Machines/ml-1m/movies.dat',
                     sep='::',header=None,engine='python',encoding='latin-1')
# movie names have ',', separator is '::', movie names may have special character so encoder 'utf-8' not used
# movies.head(3) # 4000 movies
movies=movies.rename(columns = {0:'Movie_ID'})
movies=movies.rename(columns = {1:'Movie_Title'})
movies=movies.rename(columns = {2:'Movie_Genre'})
movies.head(3)

,Movie_ID,Movie_Title,Movie_Genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


In [9]:
## import dataset:
users = pd.read_csv('C:/Users/anurag.trivedi/2. Data/5. Boltzmann Machines/Boltzmann_Machines/ml-1m/users.dat',
                     sep='::',header=None,engine='python',encoding='latin-1')
users.head(3) # 6000 users; ID, Gender, Age, Job Code, Zip Code
users=users.rename(columns = {0:'User_ID'})
users=users.rename(columns = {1:'Gender'})
users=users.rename(columns = {2:'Age'})
users=users.rename(columns = {3:'Job_Code'})
users=users.rename(columns = {4:'Zip_Code'})
users.head(3)

,User_ID,Gender,Age,Job_Code,Zip_Code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117


In [10]:
## import dataset:
ratings = pd.read_csv('C:/Users/anurag.trivedi/2. Data/5. Boltzmann Machines/Boltzmann_Machines/ml-1m/ratings.dat',
                     sep='::',header=None,engine='python',encoding='latin-1')
# ratings.head(3) # 1,000,000 ratings; User, Movie, Rating, timestamp when user rated movie
ratings=ratings.rename(columns = {0:'User_ID'})
ratings=ratings.rename(columns = {1:'Movie_ID'})
ratings=ratings.rename(columns = {2:'Rating'})
ratings=ratings.rename(columns = {3:'Time_Stamp'})
ratings.head(3)

,User_ID,Movie_ID,Rating,Time_Stamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968


#######################################################################################
############################### Data Pre-Processing ###################################
#######################################################################################

In [12]:
## reduce Train-Univ size (for faster execution)
# remove users
ratings_small = ratings.head(300000)
ratings_small # 1779 users, ~4000 movies
ratings = ratings_small

In [13]:
## Normal train test split
# 'same user same set' will not work for recomm engine because a user should have watched some movies for recomm to generate

# random sampling
np.random.seed(4)
msk = np.random.rand(len(ratings)) < 0.75                        
Train = ratings[msk] # generates random sample of nearly 75%        
Test = ratings[~msk]

# drop Time_Stamp as not required
Train = Train.drop(['Time_Stamp'], axis=1)
Test = Test.drop(['Time_Stamp'], axis=1)

# save Train & Test as csv
Train.to_csv('C:/Users/anurag.trivedi/2. Data/5. Boltzmann Machines/Boltzmann_Machines/Train.csv', sep=',')
Test.to_csv('C:/Users/anurag.trivedi/2. Data/5. Boltzmann Machines/Boltzmann_Machines/Test.csv', sep=',')

In [14]:
## converting train-test to np array
Train_arr = np.array(Train, dtype='int')
Test_arr = np.array(Test, dtype='int')

In [15]:
## creating Train & Test user-movie 2d array - list of lists, a row represents list of a user with all movie ratings
# list of lists is required to create train/test torch tensors for NN
# each matrix will have all train-test users and movies as rows and columns respectively
# If user did not rate a movie, 0 will be put there

# getting the number of user and movies
frames = [Train, Test]
Train_Test_concat = pd.concat(frames) # 300,000 rows 
nb_users = Train_Test_concat.iloc[:,0].nunique() # total no. of users = 1779
nb_movies = Train_Test_concat.iloc[:,1].nunique() # total no. of movies = 3569

# function to create list of lists for train/test - same structure of all_users*all_movies

Train_Test_concat = Train_Test_concat.sort_values(['User_ID','Movie_ID'], ascending=[True,True])

def convert(Data):
    # create dataframe of user-movie(row*col) -> convert to list of list -> convert to torch tensor
    Data_list = Train_Test_concat[['User_ID', 'Movie_ID']]
    Data_list = pd.merge(Data_list, Data, how='left', on=['User_ID', 'Movie_ID'])
    # Train_list/Test _list # 300,000 rows
    
    Data_list['Rating'].fillna(0, inplace=True) # replace NA with 0 - did not watch
    # Train_list/Test_list # ratings will be correct only w.r.t. train/test set
    # Transpose
    Data_list = Data_list.pivot_table(index=['User_ID'],columns=['Movie_ID'],values='Rating',aggfunc='first',fill_value=0)
    
    # creating list of lists
    Temp = []
    for i in range(len(Data_list.index)):
        Temp1 = []
        for j in range(len(Data_list.columns)):
            Temp1.append(Data_list.iloc[i,j])
        Temp.append(Temp1)
    return Temp

Train_list = convert(Train) 
Test_list = convert(Test) 
# Temp # list of list is created with 1779 lists and 3569 values in each list

# check the structure using pd df
# Temp2 = pd.DataFrame(Train_list)
# Temp2 # 1779 × 3569
# Temp2 = pd.DataFrame(Test_list)
# Temp2 # 1779 × 3569

In [16]:
## converting data into torch tensors
# torch tensors are basically multi-dimensional arrays with same datatype, but compatible to torch
# author says NN with torch tensors performed better than NN with tensorflow  tensors (for BM and Autoencoders)
Train_torchtensor = torch.FloatTensor(Train_list)
Test_torchtensor = torch.FloatTensor(Test_list)

In [17]:
# converting ratings to like/not-like
Train_torchtensor[Train_torchtensor == 0] = -1 # changing not seen movie to -1
Train_torchtensor[Train_torchtensor == 1] = 0 # changing 1/2 rating movie to 0
Train_torchtensor[Train_torchtensor == 2] = 0 # changing 1/2 rating movie to 0
Train_torchtensor[Train_torchtensor >= 3] = 1 # changing 3/4/5 rating movie to 1

Test_torchtensor[Test_torchtensor == 0] = -1 # changing not seen movie to -1
Test_torchtensor[Test_torchtensor == 1] = 0 # changing 1/2 rating movie to 0
Test_torchtensor[Test_torchtensor == 2] = 0 # changing 1/2 rating movie to 0
Test_torchtensor[Test_torchtensor >= 3] = 1 # changing 3/4/5 rating movie to 1

#######################################################################################
###################### Creating architecture of Neural Network ########################
#######################################################################################

In [19]:
## creating NN architecture - in pytorch
class RBM(): # always in caps
    def __init__(self, nv, nh): # initialization function to create objects; nv=no of visible node; nh=no of hidden nodes
        # nv = no of visible nodes = no of variables = no of movies
        # when object created, this function is initialized automatically
        self.W = torch.randn(nh, nv) # nh*nv are number of weights to be initialized by this nh*nv torch tensor
        # weights should be random normal distribution values(hence randn), mean 0 and sd 1
        self.a = torch.randn(1, nh) # initializing bias for hidden nodes given visible nodes(talking in PGM Model language)
        self.b = torch.randn(1, nv) # initializing bias for visible nodes given hidden nodes
    def sample_h(self, x): # Gibb's sampling function: will generate probabilities of all hidden nodes given visible nodes
        # x represents the visible neurons/visible vector in the probabilities P(h/v)
        wx = torch.mm(x, self.W.t()) # t() is transpose as W is wt matrix of P(v/h); mm=matrix multiplication
        activation = wx + self.a.expand_as(wx) # activation is linear_comb=input*weight+bias; expand_as(wx) applies bias to all 
        # input vectors in mini-batch
        p_h_given_v = torch.sigmoid(activation) # say a hidden neuron represents action movie, and Die Hard is input then this
        # neuron will get activated
        # if 100 neurons in hidden nodes, p_h_given_v represents 100 values vector of probabilities for these neurons
        return p_h_given_v, torch.bernoulli(p_h_given_v)
        # Bernoulli's Sampling-used to select 0/1(activation) as output for each hidden node; if p_h_given_v of a hidden node is
        # 0.7, then Bernoulli's Sampling generates a rand prob between 0 and 1, if rand prob is 0.8 then output 1(as > 0.7),
        # if rand prob 0.5, then output 0(as < 0.7)
    def sample_v(self, y): # Gibb's sampling function: will generate probabilities of all visible nodes given hidden nodes
        # y represents the hidden neurons/hidden vector in the probabilities V(v/h)
        wy = torch.mm(y, self.W)
        activation = wy + self.b.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)
    def train(self, v0, vk, ph0, phk): # v0 & vk - 2D input/visible nodes values and visible nodes values after k C.Divergence
        # ph0, phk are P(h0/v0) and P(hk/vk)
        # self.W += torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk) # LHS and RHS not compatible
        # 'print(rbm.W.size())'=100,3569--print((torch.mm(v0.t(), ph0)-torch.mm(vk.t(), phk)).size())=3569,100--t() required
        self.W += ( torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk) ).t() # this line is diff than Udemy course
        self.b += torch.sum((v0 - vk), 0) # 0 added to keep b as tensor of 2 dimensions
        self.a += torch.sum((ph0 - phk), 0)

# with this class, many objects/models can be created with different configurations - can also add learning rate

In [34]:
## creating RBM object/model
nv = len(Train_torchtensor[0]) # nb_movies
nh = 100 # no. of hidden nodes - say hundred features of movies to be identified - P. Tuning(100,200,)
# lr: can be added in above function; currently default
batch_size = 50 # weights update after these many steps - P. Tuning(50,100,200)
rbm = RBM(nv, nh) # creating RBM object/model
# there maybe class for 'adam', 'rmsprop' etc..maybe check

In [35]:
## training/opt the RBM
np.random.seed(8)
nb_epoch = 20 # P. Tuning(10,20)
for epoch in range(1, nb_epoch+1):
    train_loss = 0
    s = 0. # float # to normalize the train_loss by dividng with this counter after every batch
    for id_user in range(0, nb_users-batch_size, batch_size):
        vk = Train_torchtensor[id_user:id_user+batch_size] # input; vk initialized with initial values
        v0 = Train_torchtensor[id_user:id_user+batch_size] # target is v0
        ph0,_ = rbm.sample_h(v0) # ,_ returns 1st value of function, p_v_given_h
        for k in range(10): # k is number of random walks of contrastive divergence; no of times each input walks randomly
            # P. Tuning(10,20)
            _,hk = rbm.sample_h(vk) # _, returns 2nd value of function, bernoulli
            _,vk = rbm.sample_v(hk) # these 2 steps will happen 10 times(random walks) for the whole batch(1 row a time)
            vk[v0<0] = v0[v0<0]
        phk,_ = rbm.sample_h(vk) # calculated after k CDs
        rbm.train(v0, vk, ph0, phk) # weights/biases update after each batch completes random walk/Gibb's Sampling
        train_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0])) # Mean Absolute Diff (RMSE to be used in Autoencoders)
        # train_loss += np.sqrt(torch.mean((v0[v0>=0] - vk[v0>=0])**2)) # RMSE here
        # here vk is the value of visible nodes after all iterations in an epoch
        s += 1.
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

## validation set also required for train-test approach
## understanding train_loss of 0.2328 - means nearly 3/4 predictions are correct on traning set - (1st iter)
## (1st iter)-(nh=100,batch_size=100,nb_epoch=10,random_walks=10) - 0.2332
## (2nd iter)-(nh=200,batch_size=100,nb_epoch=10,random_walks=10) - 0.2337
## (3rd iter)-(nh=100,batch_size=100,nb_epoch=20,random_walks=10) - 0.2332
## (4th iter)-(nh=100,batch_size=50,nb_epoch=20,random_walks=10) - 0.2331**
## (5th iter)-(nh=100,batch_size=100,nb_epoch=20,random_walks=15) - 0.2334
## (6th iter)-(nh=100,batch_size=100,nb_epoch=20,random_walks=15) - 0.2334


epoch: 1 loss: tensor(0.2772)
epoch: 2 loss: tensor(0.2373)
epoch: 3 loss: tensor(0.2331)
epoch: 4 loss: tensor(0.2278)
epoch: 5 loss: tensor(0.2324)
epoch: 6 loss: tensor(0.2330)
epoch: 7 loss: tensor(0.2332)
epoch: 8 loss: tensor(0.2331)
epoch: 9 loss: tensor(0.2331)
epoch: 10 loss: tensor(0.2325)
epoch: 11 loss: tensor(0.2319)
epoch: 12 loss: tensor(0.2328)
epoch: 13 loss: tensor(0.2323)
epoch: 14 loss: tensor(0.2323)
epoch: 15 loss: tensor(0.2327)
epoch: 16 loss: tensor(0.2326)
epoch: 17 loss: tensor(0.2333)
epoch: 18 loss: tensor(0.2326)
epoch: 19 loss: tensor(0.2327)
epoch: 20 loss: tensor(0.2331)


In [44]:
## predicting on test data - calculating accuracy / test_loss

# approach:
# train,test are with same structure; all vacant(rating happened, but not present in train) ratings in train will be populated
# then test ratings will be compared for accuracy
np.random.seed(11)
test_loss = 0
s = 0. 
for id_user in range(nb_users): # rather than batch, for test we do this for each input
    v = Train_torchtensor[id_user:id_user+1] # input; train set input will activate all required neurons; pred to happen on train data only
    vt = Test_torchtensor[id_user:id_user+1] # target to be compared with
    # ph0,_ = rbm.sample_h(v0) # required only for training, not testing
    # for k in range(1): # model is trained, so in 1 CD we can get the output in visible nodes, so for loop not required
    if len(vt[vt>=0]) > 0: # ratings prediction will happen for rows where some rating is present
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h) # only step of blind walk-random walks happen in training; single step while generating predictions is referred as blind walk because model strives to be accurate like a blind person walking
        # vk[v0<0] = v0[v0<0] # required only for training
        # phk,_ = rbm.sample_h(vk) # required only for training
        # rbm.train(v0, vk, ph0, phk) # required only for training
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0])) # loss calculated only for present ratings present in test
        # test_loss += np.sqrt(torch.mean((vt[vt>=0] - v[vt>=0])**2)) # RMSE here
        s += 1.
        # test_loss and s counter brought under 'if' because they need to update only if some rating is present in test row
print('test loss: '+str(test_loss/s)) # epochs removed

# test loss: tensor(0.2171) is like 21.71% error, which is good - (1st iter)
## (1st iter) - 0.2171
## (4th iter) - 0.2011

test loss: tensor(0.2011)


In [ ]:
## Error Metric and Activation Function:
# MAE used here because of 0/1 predictions; Confusion Matrix can also be used
# MAE may not be the best to optimize; can optimize using RMSE but report MAE
## Activation Fn - Sigmoid (same for autoencoders)
# Why Sigmoid - Maybe because only 1 hidden layer iteration(forward pass) before weight updation, 
#               so Relu's benefits of faster weight update and lesser risk of Van. Gradient do not apply
#               Hence a more complex(non-linear) activation function used


In [ ]:
## predicting on test data - calculating accuracy / test_loss
## & creating an output with all recommendations

# this code needs to be written - will be similar to previous para

vt # shows how output looks (1/0)

In [ ]:
## also above code should be re-written with 2 vald sets, and auto parameter tuning

#######################################################################################
################### Stacked Autoencoders - Scoring on new data ######################
#######################################################################################

In [ ]:
# 1. Try to find existing packages
# 2. Custom approach:
    # find existing user closest to new user using RMSE(not cosine sim here)
    # use this existing user's recomm for the new user